In [7]:
import utils.common
import evaluation
import importlib
import numpy as np
import time
from models.ConstraintAutoRec import ConstraintAutoRec 
import models.NeuralLogicRec
import tensorflow as tf
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.NeuralLogicRec)

ml_small = utils.common.ml_small

# ev = evaluation.Evaluation(ml_small)

In [8]:
nlr = models.NeuralLogicRec.NLR(ml_small['user'], ml_small['dimensions'], epochs=3, embedding_dim =16, batch_size=1200)
nlr.train(utils.common.load_dataset(ml_small), ml_small['train']['records'])

Epoch #0 Loss at step 150: -3.4465, time: 13.523
Epoch #1 Loss at step 150: -5.5452, time: 11.185
Epoch #2 Loss at step 150: -7.6357, time: 11.023


In [18]:
print(np.flip(np.argsort(nlr.predict_single_user(3))))
print(nlr.predict_single_user(1))
# nlr.model(np.array(1), np.array(1))

[10378  3446  3464 ...  6915  6914     0]
[1. 1. 1. ... 1. 1. 1.]
